In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
tickers = pd.read_csv("data/stock_etf_tickers.csv")
# tickers = list(tickers.Ticker)
tickers = ['MSFT', "AAPL", "NVDA", "AMZN"]

In [12]:
from src.DataCollector import DataCollector

dc = DataCollector(tickers, "2017-01-01", "2017-12-31")
rt = dc.get_returns()
rt

[*********************100%***********************]  4 of 4 completed


Ticker,AAPL,AMZN,MSFT,NVDA
Date,,,,
2017-01-04,-0.001119,0.004657,-0.004474,0.023331
2017-01-05,0.005085,0.030732,0.000000,-0.025386
2017-01-06,0.011148,0.019912,0.008668,0.013368
2017-01-09,0.009159,0.001168,-0.003183,0.040543
2017-01-10,0.001008,-0.001280,-0.000319,-0.007550
...,...,...,...,...
2017-12-22,0.000000,-0.005448,0.000117,-0.003165
2017-12-26,-0.025370,0.007190,-0.001286,0.011113
2017-12-27,0.000176,0.004674,0.003630,-0.001367


In [37]:
ff = pd.read_csv("data/ff_daily.csv")
ff['Date'] = pd.to_datetime(ff['Date'], format = "%Y%m%d")
ff.set_index('Date', inplace = True)
common = rt.index.intersection(ff.index)
ff = ff.loc[common]
mkt = ff['Mkt-RF'] / 100
mkt

Date
2017-01-04    0.0079
2017-01-05   -0.0021
2017-01-06    0.0029
2017-01-09   -0.0037
2017-01-10    0.0016
               ...  
2017-12-22   -0.0007
2017-12-26   -0.0007
2017-12-27    0.0005
2017-12-28    0.0022
2017-12-29   -0.0057
Name: Mkt-RF, Length: 250, dtype: float64

In [45]:
import statsmodels.api as sm

alphas = {}
betas = {}
resvars = {}

for stock in tickers:
    y = rt[stock]
    Xdesign = sm.add_constant(mkt)
    model = sm.OLS(y, Xdesign).fit()

    alphas[stock]  = model.params["const"]
    betas[stock]   = model.params["Mkt-RF"]
    resvars[stock] = model.resid.var(ddof=0)

summary = pd.DataFrame({
    "alpha"     : alphas,
    "beta"      : betas,
    "resid_var" : resvars
}).T

summary

,MSFT,AAPL,NVDA,AMZN
alpha,0.000507,0.000723,0.001320,0.000973
beta,1.157999,1.201533,2.082481,1.151410
resid_var,0.000059,0.000093,0.000549,0.000147


In [104]:
print(rt.shape)
print(rt.index[:3], "← index")
print(rt.columns[:3], "← columns")

market = mkt.to_numpy()
x = rt.to_numpy().T
m_bar = market.mean()
x_bar = x.mean(axis = 1, keepdims = True)
x_bar

N, T = x.shape
assert len(tickers) == N
print(tickers, rt.columns.to_list())

m_c = market - m_bar
x_c = x - x_bar

beta_hat = (x_c @ m_c) / (m_c @ m_c)
beta_hat

alpha_hat = x_bar.flatten() - beta_hat * m_bar
alpha_hat

eps = x - alpha_hat[:, None] - beta_hat[:, None] * market
resid_var  = (eps ** 2).mean(axis=1)
resid_var

results = pd.DataFrame(
    {"alpha": alpha_hat,
     "beta":  beta_hat,
     "resid_var": resid_var},
    index=rt.columns.to_list()
)
results

(250, 4)
DatetimeIndex(['2017-01-04', '2017-01-05', '2017-01-06'], dtype='datetime64[ns]', name='Date', freq=None) ← index
Index(['AAPL', 'AMZN', 'MSFT'], dtype='object', name='Ticker') ← columns
['MSFT', 'AAPL', 'NVDA', 'AMZN'] ['AAPL', 'AMZN', 'MSFT', 'NVDA']


,alpha,beta,resid_var
AAPL,0.000723,1.201533,0.000093
AMZN,0.000973,1.151410,0.000147
MSFT,0.000507,1.157999,0.000059
NVDA,0.001320,2.082481,0.000549


In [49]:
import numpy as np

T = 10
N = 4

market = 0.15 * np.random.randn(T)

true_betas = np.array([1.0, 0.8, 1.2, 0.5])
true_alphas = np.array([0.001, -0.0005, 0.0007, 0.0])
res_sd = np.array([0.10, 0.08, 0.12, 0.09])

X = true_alphas[:, None] + true_betas[:, None] * market + res_sd[:, None] * np.random.randn(N, T)
X

array([[-0.0979157 , -0.02554747,  0.12066443, -0.11500772, -0.04471057,
         0.35143176,  0.0890803 ,  0.0430127 ,  0.01592689,  0.3522757 ],
       [-0.31609393, -0.06802273,  0.10060396, -0.00764949, -0.10444228,
         0.30442484,  0.03838578,  0.08427627,  0.06994382,  0.10384034],
       [-0.48921728, -0.35371742,  0.21130517, -0.17983523, -0.2910925 ,
         0.42164161,  0.05676357,  0.03716657,  0.16461968,  0.29032658],
       [-0.25466967,  0.08526089,  0.14645724, -0.02747931, -0.16622787,
         0.32256134,  0.0922575 ,  0.13252063, -0.05082654,  0.09322504]])